In [1]:
%matplotlib inline
from __future__ import division, print_function
from future.builtins import object, super, range
from future.utils import with_metaclass
from abc import ABCMeta, abstractmethod
import hashlib
import numpy as np
import scipy.optimize as spop
from fatiando.utils import safe_dot

In [ ]:
class NonLinearModel(with_metaclass(ABCMeta)):
    def __init__(self, nparams, misfit='l2', optimizer='nelder-mead'):
        self.nparams = nparams
        self.islinear = False
        self.regularization = []
        self.misfit = None
        self.optimizer = None
        self.set_misfit(misfit)
        self.set_optimizer(optimizer)
    
    @abstractmethod
    def predict(self, args):
        "Return data predicted by self.p_"
        pass
        
    def set_optimizer(self, optimizer):
        "Configure the optimization"
        
        return self
    
    def set_misfit(self, misfit):
        "Pass a different misfit function"
        if misfit == 'l2':
            self.misfit = L2NormMisfit()
        else:
            self.misfit = misfit        
        return self
    
    def add_regularization(self, regul_param, regul):
        "Use the given regularization"
        self.regularization.append([regul_param, regul])
        return self
    
    def _make_partial(self, args, func):
        def partial(p):
            backup = self.p_
            self.p_ = p
            res = getattr(self, func)(*args)
            self.p_ = backup
            return res
        return partial
            
    def fit(self, args, data, weights=None):
        "Fit the model to the given data"
        misfit_args = dict(data=data, predict=self._make_partial(args, 'predict'),
                           weights=weights, islinear=self.islinear)
        if hasattr(self, 'jacobian'):
            misfit_args['jacobian'] = self._make_partial(args, 'jacobian')
        misfit = self.misfit(**misfit_args)            
        objective = Objective([[1, misfit]] + self.regularization)
        self.p_ = self.optimizer.minimize(objective) # the estimated parameter vector
        return self

In [ ]:
class LinearModel(NonLinearModel):
    pass

In [ ]:
class L2NormMisfit(object):
    def __init__(self, data, predict, islinear, jacobian=None, weights=None):
        self.data = data
        self.predict = predict
        self.jacobian = jacobian
        self.weights = weights
        self.islinear = islinear
        self.cache = {'predict': {'hash':None, 'value': None},
                      'jacobian': {'hash':None, 'value': None}}
        
    def from_cache(self, p, func):
        if func == 'jacobian' and self.islinear:
            if self.cache[func]['value'] is None:
                self.cache[func]['value'] = getattr(self, func)(p)
        else:
            new_hash = hashlib.sha1(p).hexdigest()
            old_hash = self.cache[func]['hash']
            if old_hash is None or old_hash != new_hash:
                self.cache[func]['hash'] = new_hash
                self.cache[func]['value'] = getattr(self, func)(p)
        return self.cache[func]['value']            
    
    def value(self, p):
        pred = self.from_cache(p, 'predict')
        residuals = self.data - pred
        if self.weights is None:
            return np.linalg.norm(residuals, ord=2)**2
        else:
            return safe_dot(residuals.T, safe_dot(weights, residuals))
        
    def gradient(self, p):
        jac = self.from_cache(p, 'jacobian')
        pred = self.from_cache(p, 'predict')
        residuals = self.data - pred
        if weights is None:
            grad = -2*safe_dot(jac.T, residuals)
        else:
            grad = -2*safe_dot(jac.T, safe_dot(weights, residuals))
        return self._grad_to_1d(grad)
    
    def _grad_to_1d(self, grad):
        # Check if the gradient isn't a one column matrix
        if len(grad.shape) > 1:
            # Need to convert it to a 1d array so that hell won't break
            # loose
            grad = np.array(grad).ravel()
        return grad
    
    def gradient_at_null(self):
        jac = self.from_cache(p, 'jacobian')
        if weights is None:
            grad = -2*safe_dot(jac.T, self.data)
        else:
            grad = -2*safe_dot(jac.T, safe_dot(weights, self.data))
        return self._grad_to_1d(grad)        
    
    def hessian(self, p):
        jac = self.from_cache(p, 'jacobian')
        if weights is None:
            return 2*safe_dot(jac.T, jac)
        else:
            return 2*safe_dot(jac.T, safe_dot(weights, jac))

In [ ]:
class Objective(object):
    def __init__(self, components):
        self.components = components
    
    def value(self, p):
        return np.sum(lamb*comp.value(p) 
                       for lamb, comp in self.components)
    
    def gradient(self, p):
        return np.sum(lamb*comp.gradient(p) 
                       for lamb, comp in self.components)
    
    def hessian(self, p):
        return np.sum(lamb*comp.hessian(p) 
                       for lamb, comp in self.components)

In [ ]:
class GaussNewton(object):
    def __init__(self, initial, tol, precondition):
        pass
    def minimize(self, objective):
        pass
    
class ScipyOptimizer(object):
    def __init__(self, method, **kwargs):
        self.method = self._is_valid(method)
        self.args = kwargs
        
    def _is_valid(self, method):
        return method
    
    def minimize(self, objective):
        pass    